In [4]:
import ipywidgets as widgets
from IPython.display import display, HTML
from collections import defaultdict
import random
import io
import base64

# 🎨 Style-enhanced text input
sentence_input = widgets.Textarea(
    value="The stars shine brightly in the midnight sky and the world sleeps peacefully.",
    description="🌟 Your Sentence:",
    layout=widgets.Layout(width='100%', height='100px'),
    style={'description_width': 'initial'}
)

# 🔘 Start word dropdown (auto-filled)
start_word_dropdown = widgets.Dropdown(
    options=[],
    description="🚀 Start Word:",
    style={'description_width': 'initial'}
)

# 🔢 Output length text input
length_input = widgets.Text(
    value="30",
    description="📝 Output Length:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

# 🎉 Generate button with style
generate_button = widgets.Button(
    description="✨ Generate Magic ✨",
    button_style='info',
    layout=widgets.Layout(width='200px')
)

# 📦 Output area
output_area = widgets.Output()

# 📥 Fancy download link
def get_download_link(text, filename="generated.txt"):
    b = io.BytesIO()
    b.write(text.encode())
    b.seek(0)
    b64 = base64.b64encode(b.read()).decode()
    return HTML(f'''
        <div style="margin-top:10px;">
            <a download="{filename}" href="data:text/plain;base64,{b64}"
               style="font-size:16px; color:#fff; background:#4CAF50; padding:8px 12px;
                      text-decoration:none; border-radius:5px;">
               📥 Download Your Text
            </a>
        </div>
    ''')

# 🔗 Build Markov chain
def build_chain(text):
    words = text.lower().split()
    chain = defaultdict(list)
    for current, next_ in zip(words[:-1], words[1:]):
        chain[current].append(next_)
    return chain, words

# 📝 Generate text
def generate_text(chain, start_word, length):
    word = start_word
    output = [word]
    for _ in range(length - 1):
        next_words = chain.get(word)
        if not next_words:
            break
        word = random.choice(next_words)
        output.append(word)
    return ' '.join(output)

# 🔁 Update dropdown when sentence changes
def update_start_word_dropdown(change):
    _, words = build_chain(sentence_input.value)
    unique_words = sorted(set(words))
    start_word_dropdown.options = unique_words if unique_words else [""]

sentence_input.observe(update_start_word_dropdown, names='value')

# ▶️ On click
def on_generate_clicked(b):
    output_area.clear_output()
    chain, _ = build_chain(sentence_input.value)
    start_word = start_word_dropdown.value
    try:
        length = int(length_input.value)
    except ValueError:
        with output_area:
            print("⚠️ Please enter a valid number for output length.")
        return
    result = generate_text(chain, start_word, length)
    with output_area:
        display(HTML(f"<h4>🌈 Generated Text (Start: <i>{start_word}</i>, Length: {length})</h4>"))
        print(result)
        display(get_download_link(result))

generate_button.on_click(on_generate_clicked)

# 📦 Display UI
display(
    widgets.VBox([
        sentence_input,
        widgets.HBox([start_word_dropdown, length_input]),
        generate_button,
        output_area
    ])
)
update_start_word_dropdown(None)